## PROJECT-3. Анализ воронки

*Работу подготовила: Машковцева Дарья.*

ЦЕЛЬ

Исследовать поведение пользователей в обновлённом приложении.

ЗАДАЧИ

В рамках проверки гипотез вам необходимо решить три задачи:
1. Определить, насколько обучение сокращает время прохождения этапов игры.
2. Доказать, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.
3. Определить, насколько прозрачен процесс взаимодействия с игрой.

КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)
1. Сравнить время прохождения различных этапов для пользователей, которые завершили обучение, и пользователей, не начинавших обучение. Если показатель отличается, выяснить, насколько.
2. Проверить, существует ли зависимость между вероятностью оплаты вопросов и количеством обучений, которые начинал или завершал пользователь. 
3. Выяснить, как часто пользователи начинают обучение после выбора уровня сложности. (Это позволит оценить прозрачность процесса взаимодействия с игрой: если пользователи после выбора уровня сложности обращаются к обучению, значит, работа с приложением непонятна.)

In [3]:
import pandas as pd

events_data = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_data = pd.read_csv('data/purchase.csv', sep=',')

*Подготовка данных*

In [4]:
cond = (events_data.start_time>='2018-01-01') & (events_data.start_time<'2019-01-01') & (events_data.event_type=='registration')
registered = events_data[cond]['user_id'].to_list() # список пользователей, зарег. в 2018
events_df = events_data[events_data.user_id.isin(registered)]
events_df.start_time = pd.to_datetime(events_df.start_time, format='%Y-%m-%dT%H:%M:%S')

cond2 = (purchase_data.event_datetime>='2018-01-01') & (purchase_data.event_datetime<'2019-01-01')
registered2 = purchase_data[cond2]['user_id'].to_list()
purchase_df = purchase_data[purchase_data['user_id'].isin(registered)]
purchase_df.event_datetime = pd.to_datetime(purchase_df.event_datetime, format='%Y-%m-%dT%H:%M:%S')

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


*Рассчитываем количество пользователей на каждом этапе.*

In [5]:
registered_users_count = events_df[events_df["event_type"] == "registration"]["user_id"].nunique()
tutorial_start_users_count = events_df[events_df["event_type"] == "tutorial_start"]["user_id"].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count

tutorial_finish_users_count = events_df[events_df["event_type"] == "tutorial_finish"]["user_id"].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count

level_choice_users_count = events_df[events_df["event_type"] == "level_choice"]["user_id"].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count

training_choice_users_count = events_df[events_df["event_type"] == "pack_choice"]["user_id"].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count

paying_users_count = purchase_df["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count

purchase_rate = paying_users_count / registered_users_count

*Объединяем таблицы*

In [32]:
purchase_df['event_type'] = 'purchase'
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})
purchase_df = purchase_df.rename(columns={"event_datetime": "start_time"})
total_events_df = pd.concat([events_df,purchase_df],sort=False)

total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')
total_events_df

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaN
...,...,...,...,...,...,...,...,...
68554,NaN,purchase,NaN,2019-01-02 03:48:19,NaN,47498,18441.0,100.0
68555,NaN,purchase,NaN,2019-01-02 23:26:26,NaN,47647,18442.0,150.0
68556,NaN,purchase,NaN,2019-01-03 00:36:36,NaN,47554,18443.0,50.0
68557,NaN,purchase,NaN,2019-01-04 12:51:41,NaN,47742,18448.0,50.0


*Выявляем наиболее популярные пути с оплатой*

In [33]:
user_path_df = (total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index())

user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))

user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)

*Определяем время между этапами*

In [37]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df = registration_df[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})

tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']
tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values("start_time").drop_duplicates("user_id")
tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[["user_id", "tutorial_id", "start_time"]].rename(columns={"start_time": "tutorial_start_time"})

merged_df = registration_df.merge(tutorial_start_df_wo_duplicates, on="user_id", how="inner")
merged_df["timedelta"] = (merged_df["tutorial_start_time"] - merged_df["registration_time"])
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time,timedelta
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40,0 days 13:40:15
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11,0 days 11:11:01
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51,0 days 03:06:04
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40,0 days 01:26:33
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36,0 days 00:47:35


*Определяем время обучения*

In [8]:
tutorial_finish_df = total_events_df[total_events_df["event_type"] == "tutorial_finish"]

first_tutorial_ids = tutorial_start_df_wo_duplicates["tutorial_id"].unique()
tutorial_finish_df = tutorial_finish_df[tutorial_finish_df["tutorial_id"].isin(first_tutorial_ids)]

tutorial_finish_df = tutorial_finish_df[["user_id", "start_time"]].rename(columns={"start_time": "tutorial_finish_time"})
merged_df_2 = tutorial_start_df_wo_duplicates.merge(tutorial_finish_df, on="user_id", how="inner")
merged_df_2["timedelta"] = (merged_df_2["tutorial_finish_time"] - merged_df_2["tutorial_start_time"])

print(merged_df_2["timedelta"].mean())
print(merged_df_2["timedelta"].describe())

0 days 00:03:53.174160732
count                         9830
mean     0 days 00:03:53.174160732
std      0 days 00:01:44.803217992
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%                0 days 00:05:08
max                0 days 00:10:06
Name: timedelta, dtype: object


*Определяем время между registration и level_choice*

In [9]:
level_choice_df = total_events_df[total_events_df["event_type"] == "level_choice"]
level_choice_df = level_choice_df[["user_id", "start_time"]].rename(columns={"start_time": "level_choice_time"})
merged_df_3 = registration_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_3["timedelta"] = (merged_df_3["level_choice_time"] - merged_df_3["registration_time"])

print(merged_df_3["timedelta"].mean())
print(merged_df_3["timedelta"].describe())

0 days 07:10:19.169863342
count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


*Ищем пользователей, прошедших обучение хотя бы раз*

In [47]:
users_with_finished_tutorial = total_events_df[total_events_df["event_type"] == "tutorial_finish"]["user_id"].unique()

*Ищем пользователей, которые начали, но не закончили обучение*

In [45]:
users_with_started_tutorial = total_events_df[total_events_df["event_type"] == "tutorial_start"]["user_id"].unique()
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = (set_users_with_started_tutorial.difference(set(users_with_finished_tutorial)))

print(len(set_users_not_finished_but_started_tutorial))

1608


*Ищем пользователей, ни разу не начинавших обучение*

In [16]:
all_users = total_events_df["user_id"].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(set(users_with_started_tutorial))

print(len(set_users_not_started_tutorial))

8068


*Считаем пользователей, которые завершили обучение и совершили оплату*

In [17]:
purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_finished_tutorial)]
percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / len(users_with_finished_tutorial)

*Считаем средний размер платежа разных групп*

In [18]:
purchase_df_2 = purchase_df[purchase_df["user_id"].isin(set_users_not_finished_but_started_tutorial)]
#print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(set_users_not_finished_but_started_tutorial)
purchase_df_2['amount'].mean()

104.9618320610687

*Определяем, насколько обучение сокращает время прохождения этапов игры*

*Ищем пользователей, прошедших обучение хотя бы раз*

In [49]:
users_with_finished_tutorial

array([27835, 27836, 27833, ..., 47753, 47755, 47756], dtype=int64)

In [48]:
users_with_finished_tutorial = total_events_df[total_events_df["event_type"] == "tutorial_finish"]["user_id"].unique()
purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_finished_tutorial)]
purchase_df_1

,purchase_id,user_id,start_time,amount,event_type
1171,16845,27845,2018-01-03 18:53:43,100,purchase
1175,16849,27910,2018-01-07 12:11:34,100,purchase
1176,16850,27940,2018-01-07 13:16:41,200,purchase
1178,16852,27981,2018-01-07 23:20:25,50,purchase
1179,16853,27969,2018-01-08 01:18:23,100,purchase
...,...,...,...,...,...
2767,18441,47498,2019-01-02 03:48:19,100,purchase
2768,18442,47647,2019-01-02 23:26:26,150,purchase
2769,18443,47554,2019-01-03 00:36:36,50,purchase
2774,18448,47742,2019-01-04 12:51:41,50,purchase


In [42]:
tutorial_finish_df = total_events_df[total_events_df['event_type'] == 'tutorial_finish']
tutorial_finish_df_wo_duplicates = tutorial_finish_df.sort_values("start_time").drop_duplicates("user_id")
tutorial_finish_df_wo_duplicates = tutorial_finish_df_wo_duplicates[["user_id"]]
tutorial_finish_df_wo_duplicates

,user_id
9,27835
11,27836
13,27833
16,27839
18,27834
...,...
66935,47747
66944,47751
66946,47753
66953,47755


In [53]:
users_with_started_tutorial = total_events_df[total_events_df["event_type"] == "tutorial_start"]["user_id"].unique()
all_users = total_events_df["user_id"].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(set(users_with_started_tutorial))

{32771,
 32780,
 32781,
 32782,
 32783,
 32785,
 32788,
 32789,
 32791,
 32792,
 32793,
 32799,
 32802,
 32814,
 32820,
 32822,
 32823,
 32826,
 32829,
 32831,
 32834,
 32837,
 32838,
 32840,
 32842,
 32846,
 32848,
 32859,
 32861,
 32862,
 32864,
 32865,
 32866,
 32867,
 32868,
 32869,
 32877,
 32884,
 32886,
 32887,
 32891,
 32893,
 32898,
 32899,
 32902,
 32909,
 32912,
 32914,
 32923,
 32926,
 32929,
 32932,
 32936,
 32939,
 32940,
 32945,
 32946,
 32948,
 32951,
 32959,
 32960,
 32964,
 32966,
 32967,
 32968,
 32969,
 32975,
 32976,
 32977,
 32978,
 32980,
 32983,
 32990,
 32991,
 32994,
 32995,
 32998,
 33000,
 33001,
 33009,
 33010,
 33013,
 33015,
 33021,
 33024,
 33025,
 33027,
 33029,
 33030,
 33031,
 33032,
 33034,
 33036,
 33037,
 33040,
 33041,
 33043,
 33045,
 33046,
 33047,
 33052,
 33053,
 33056,
 33062,
 33063,
 33064,
 33066,
 33067,
 33068,
 33070,
 33071,
 33072,
 33073,
 33076,
 33078,
 33079,
 33080,
 33083,
 33086,
 33092,
 33097,
 33100,
 33101,
 33102,
 33103,


*Определяем время прохождения между этапами для пользователей, прошедших обучение*

In [67]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df = registration_df[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})

tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']
tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values("start_time").drop_duplicates("user_id")
tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[["user_id", "tutorial_id", "start_time"]].rename(columns={"start_time": "tutorial_start_time"})

merged_df = registration_df.merge(tutorial_start_df_wo_duplicates, on="user_id", how="inner")
merged_df["timedelta_1"] = (merged_df["tutorial_start_time"] - merged_df["registration_time"])

table = merged_df.filter(['user_id', 'timedelta_1'])

tutorial_finish_df = total_events_df[total_events_df["event_type"] == "tutorial_finish"]
first_tutorial_ids = tutorial_start_df_wo_duplicates["tutorial_id"].unique()

tutorial_finish_df = tutorial_finish_df[tutorial_finish_df["tutorial_id"].isin(first_tutorial_ids)]
tutorial_finish_df = tutorial_finish_df[["user_id", "start_time"]].rename(columns={"start_time": "tutorial_finish_time"})

merged_df_2 = tutorial_start_df_wo_duplicates.merge(tutorial_finish_df, on="user_id", how="inner")
merged_df_2["timedelta_2"] = (merged_df_2["tutorial_finish_time"] - merged_df_2["tutorial_start_time"])

table = table.merge(merged_df_2.filter(['user_id', 'timedelta_2']), on="user_id", how="inner")

